In [1]:
import torch
import pandas as pd
from skimage import io, transform, color
from PIL import Image

from torchvision import models
from torchvision.io import read_image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import dataset, dataloader, Dataset
from torchvision import transforms, utils


In [2]:
torch.cuda.is_available()

True

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
import pandas as pd
from glob import glob
np.random.seed(42)
all_dirs = glob("data/tiny-imagenet-200/train/*")
dirs = np.random.choice(all_dirs, 20)
data = []
for img_dir in dirs:
    imgs_in_dir = glob(img_dir + "/images/*.JPEG")
    name = img_dir.split("/")[-1]
    for img in imgs_in_dir:
        data.append([name, img])
        
train_df = pd.DataFrame(data, columns=["name", "image"])



In [23]:
train_df.head()


,name,image
0,n04070727,data/tiny-imagenet-200/train/n04070727/images/...
1,n04070727,data/tiny-imagenet-200/train/n04070727/images/...
2,n04070727,data/tiny-imagenet-200/train/n04070727/images/...
3,n04070727,data/tiny-imagenet-200/train/n04070727/images/...
4,n04070727,data/tiny-imagenet-200/train/n04070727/images/...


In [24]:
class ImageDataset(Dataset):

    def __init__(self, df, transform=None, alexnet_feature=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        # self.root_dir = root_dir
        self.transform = transform
        self.alexnet_feature = alexnet_feature

    def __len__(self):
        return self.df.shape[0] 

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = io.imread(self.df.iloc[idx].image)
        # image = read_image(self.df.iloc[idx].image).float()
        # image = Image.open(self.df.iloc[idx].image)
        if len(image.shape) == 2:
            image = color.gray2rgb(image)
        # adds another dimension to the image channel
        name = f"{self.df.iloc[idx].name}"
        sample = {'name': name, 'image': image}
        # print("name", sample['name'], sample['image'].shape)
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        if self.alexnet_feature:
            sample['alex'] = self.alexnet_feature(sample['image'])

        return sample

In [25]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

class ToTensor(object):
    def __call__(self, image):

        # swap color axis because
        # numpy image: h x w x c
        # torch image: c x h x w
        # print(sample['name'], sample['image'].shape, sample['fix_map'].shape)
        image = image.transpose((2, 0, 1))
        return torch.from_numpy(image).float()
        # return {'name': sample['name'],
        #         'image': torch.from_numpy(image).float(),
        #         'fix_map': torch.from_numpy(fix_map).float()}


class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        self.output_size = output_size # tuple

    def __call__(self, image):

        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))
        # h and w are swapped for fix_map because for images,
        # x and y axes are axis 1 and 0 respectively
        # fix_map = fix_map * [new_w / w, new_h / h]
        return img

class GetAlexConv(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, layer_number, layer_name):
        self.layer_number = layer_number
        self.layer_name = layer_name

        self.alex = models.alexnet(pretrained=True).to(device)
        for param in self.alex.parameters():
            param.requires_grad = False
        child = list(self.alex.children())
        conv_layer = child[self.layer_number[0]][self.layer_number[1]]
        # conv_layer = child[0][10]
        conv_layer.register_forward_hook(get_activation(self.layer_name))

        # layer2 = 'maxpool5'
        # max_five = child[0][12]
        # max_five.register_forward_hook(get_activation(layer2))
        # print(conv_five.weight.size())

    def __call__(self, image):
        # return True
        # batch = np.expand_dims(image, axis=0)

        batch = image.to(device).unsqueeze(0)
        self.alex(batch)
        feature = activation[self.layer_name].squeeze(0)
        return feature


In [26]:
import torchvision.transforms.functional as fn

In [27]:

transformed_dataset = ImageDataset(train_df,
                                   transform=transforms.Compose([
                                       Rescale((227, 227)),
                                       # transforms.Resize(227),
                                       # RandomCrop(224),
                                       ToTensor(),
                                       # transforms.ToTensor(),
                                    #    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                   ]),
                                    alexnet_feature=GetAlexConv(layer_number=[0,5], layer_name='maxpool2'),
                                   )


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

net = Net().to(device)
# print(net)
# net(image).size()


In [29]:
import torch.optim as optim

trainloader = torch.utils.data.DataLoader(transformed_dataset, batch_size=64,
                                          shuffle=True, num_workers=0)


criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)


In [30]:
len(trainloader)

157

In [ ]:

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # name, image, fix_map = data
        image = data['image'].to(device)
        input_image = data['alex'].to(device)
        optimizer.zero_grad()
        print(".", end="")
        # forward + backward + optimize
        outputs = net(input_image)
        # image_resize = fn.resize(image, size=192)
        loss = criterion(outputs, image)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        # if i % 50 == 49:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 50))
        #     running_loss = 0.0

    print("epoch", epoch, running_loss / len(trainloader))

print('Finished Training')


.............................................................................................................................................................epoch 0 0.19716408060994117
......................................................................................................................................................

In [182]:
running_loss

1.3437084555625916

In [17]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook